In [1]:
import cv2
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Opencv Drawing pad to get input images 
#Same as the opencv drawing pad

In [3]:
events = [i for i in dir(cv2) if 'EVENT' in i]
#print (events)

In [4]:
def draw_number(event,x,y,flags,param):
    
    global draw
    global prev_point
    if event == cv2.EVENT_LBUTTONDOWN:     #left click pressed in
        draw = True
    elif event == cv2.EVENT_MOUSEMOVE:     #mouse movement,present location in x,y
        if draw == True and prev_point != None:
                cv2.line(draw_pad,prev_point, (x,y),(255),10)  #draws line joining prev_point to point
        prev_point = (x,y)
    elif event == cv2.EVENT_LBUTTONUP:    #left click released
        draw = False

In [5]:
draw_pad = np.zeros((512,512))
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_number)
prev_point = None

In [6]:
draw = False
while(1):
    cv2.imshow('image', draw_pad)
    if cv2.waitKey(20) & 0xFF == 13:  # ascii(enter key) = 13, press enter key to quit
        break

cv2.destroyAllWindows()

#print(draw_pad)

In [7]:
# Converting the input image to MNIST image
#Image Preprocessing starts

In [8]:
#crops the image ,removing the excess the drawpad area at the edges
def crop_image(img):
    for i in range(0,img.shape[0]):
        flag_up = np.any(img[i,:])
    
        if flag_up == True:
            up = i
            break

    for i in reversed(range(0,img.shape[0])):
        flag_down = np.any(img[i,:])
    
        if flag_down == True:
            down = i
            break
        
    for i in range(0,img.shape[1]):
        flag_left = np.any(img[:,i])
    
        if flag_left == True:
            left = i
            break

    for i in reversed(range(0,img.shape[1])):
        flag_right = np.any(img[:,i])
    
        if flag_right == True:
            right = i
            break
        
    crop = img[up:down,left:right]

    height = down - up
    width = right - left
    #cv2.imwrite('2crop.png',crop)
    return crop,height,width

#resizes image like a MNIST image
def resize_image(img,height,width):

    aspect_ratio = float(height/width)
    #type(img)
    
    if height > width:
        new_width = int(round(20 / aspect_ratio, 0))  #rounding off width with 0 decimal places,ie nearest int
        if (new_width == 0):  # rare case but minimum should be 1 pixel
            new_width = 1

        dim = (new_width, 20)   #image resized to fit in 20*20 box maintaining the aspect ratio
        im = cv2.resize(img, dim)
        #cv2.imwrite('3mini.png',im)
        wleft = 28 - new_width  # calculate vertical pozition
        #resizing to 28*28
        
        if wleft % 2 == 0:
            #adding extra pixels line after the edges to convert image to 28*28
            #equal lines added to width from top and bottom
            newImage = cv2.copyMakeBorder(im,4,4,int(wleft/2),int(wleft/2), borderType= cv2.BORDER_CONSTANT, value=0)
            
        else:
            #adding extra pixels line after the edges to convert image to 28*28
            # UN-equal lines added to width from top and bottom
            newImage = cv2.copyMakeBorder(im, 4, 4, int(round((wleft-1)/2, 0)), int(round((wleft-1)/2, 0)) +1, borderType= cv2.BORDER_CONSTANT, value=0)
            
    elif width > height:
        
        new_height = int(round(20 * aspect_ratio, 0))  # rounding off width with 0 decimal places,ie nearest int
        if (new_height == 0):  # rare case but minimum is 1 pixel
            new_height = 1
               
        dim = (20, new_height)   #image resized to fit in 20*20 box maintaining the aspect ratio
        im = cv2.resize(img, dim)
        #cv2.imwrite('3mini.png',im)
        hleft = 28 - new_height  # calculate vertical pozition
        
        if hleft % 2 == 0:
            newImage = cv2.copyMakeBorder(im, int(hleft/2), int(hleft/2), 4, 4, borderType= cv2.BORDER_CONSTANT, value=0)
        else:
            newImage = cv2.copyMakeBorder(im, int(round((hleft-1)/2, 0)), int(round((hleft-1)/2, 0)) +1, 4, 4, borderType= cv2.BORDER_CONSTANT, value=0)
            
    else: #the cases where cropped image is a square
        dim = (20,20)
        im=cv2.resize(img,dim)
        #cv2.imwrite('3mini.png',im)
        newImage = cv2.copyMakeBorder(im, 4, 4, 4, 4, borderType= cv2.BORDER_CONSTANT, value=0)
        
    #cv2.imwrite('4Preprocessed_image.png',newImage)
    return newImage


#function to combine cropping and resizing
#Complete MNIST Image Preprocessing
def preprocess(img):
    cropped_image, h, w = crop_image(img)
    #print(cropped_image.shape,type(cropped_image))
    Final_image =resize_image(cropped_image, h, w)
    
    return Final_image

In [9]:
Preprocessed_image = preprocess(draw_pad)

In [10]:
# L layer hidden network 
# Digit Recognition process starts

In [11]:
#initialize dict to store parameters

W=dict()
b=dict()
A=dict()
Z=dict()

#initializing/loading weights and biases from the files
W = np.load('W_weights.npy').item()
b = np.load('b_bias.npy').item()

In [12]:
#img = cv2.imread(Final_image,0)  # opens image in greyscale, use 1 for rgb

img = Preprocessed_image
#cv2.imwrite('preprocessed_image.png')

In [13]:
# converts image of 28*28 ,2D array to 784*1 ,1D array
data = np.asarray(img) 
data = data.reshape(784,1)
#print(data)
#print(data.shape)
#type(data)


In [14]:
#Using the saved model to get the latest(best) parameter values for drawn test case

layers_dims = [784, 150, 10]      #should be same as the trained model
L=len(layers_dims)-1
m=data.shape[1]
A[0] = data
for i in range (1,L):
    Z[i] = np.matmul(W[i], A[i-1]) + b[i]
    A[i] = np.maximum(0,Z[i])     # relu(Z[i])
    
Z[L] = np.matmul(W[L], A[L-1]) + b[L]
A[L] = np.exp(Z[L]) / np.sum(np.exp(Z[L]), axis=0)

predictions = int(np.argmax(A[L], axis=0))
print('The number should be',predictions)
#print(A[L])


The number should be 1
